In [1]:
import tensorflow as tf
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2

2024-11-13 03:01:24.633875: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 03:01:24.641294: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 03:01:24.661684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731459684.693834   16601 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731459684.701545   16601 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 03:01:24.733242: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
def detect_predict_mask(frame, face_net, mask_net):
    (h, w) = frame.shape[:2]

    # Preprocess the frame for the network
    # blob is a 4D numpy array of shape (1, height, width, channels)
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

    # Pass the blob through the network and obtain the face detections
    face_net.setInput(blob)

    # training
    # detection will have a shape of (1, 1, N, 7) where N is the number of detected faces
    # Each detection includes [batch_id, class_id, confidence, x1, y1, x2, y2]
    detection = face_net.forward() 
    print(detection.shape)

    faces, locs, preds = [], [], []

    for i in range(detection.shape[2]): # number of faces
        # the face number i has [batch_id, class_id, confidence, x1, y1, x2, y2] and index number 2 is the confidence
        # confidence is a probability score
        confidence = detection[0, 0, i, 2]
        if confidence > 0.5:
            box = detection[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            if face.size > 0:  # Ensure face area is not empty
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (224, 224))
                face = tf.keras.preprocessing.image.img_to_array(face)
                face = tf.keras.applications.mobilenet_v2.preprocess_input(face)

                faces.append(face)
                locs.append((startX, startY, endX, endY))
    
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = mask_net.predict(faces, batch_size=32)

    return (locs, preds)

In [3]:
protoTxt_path = './face_detection/deploy.prototxt'
weights_path =  './face_detection/res10_300x300_ssd_iter_140000.caffemodel'
face_net = cv2.dnn.readNet(protoTxt_path, weights_path)

mask_net = tf.keras.models.load_model('../../../mask_detector')

2024-11-13 03:01:47.827780: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
vs = VideoStream(src=0).start()

while True:
    frame = vs.read()
    if frame is None:
        print("Failed to capture frame. Check the video source.")
        break
        
    frame = imutils.resize(frame, width=400)
    (locations, predict) = detect_predict_mask(frame, face_net, mask_net)

    # loop over the detected faces
    for (box, pred) in zip(locations, predict):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_COMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

vs.stream.release()
cv2.destroyAllWindows()

(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ahmedcr7/.local/lib/python3.11/site-packages/cv2/qt/plugins"


(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97